In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from omrdatasettools.Downloader import Downloader
from omrdatasettools.OmrDataset import OmrDataset
from omrdatasettools.HomusImageGenerator import HomusImageGenerator
from omrdatasettools.MuscimaPlusPlusMaskImageGenerator import MuscimaPlusPlusMaskImageGenerator
from omrdatasettools.MuscimaPlusPlusSymbolImageGenerator import MuscimaPlusPlusSymbolImageGenerator
from omrdatasettools.AudiverisOmrImageGenerator import AudiverisOmrImageGenerator
from omrdatasettools.CapitanImageGenerator import CapitanImageGenerator
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models


The downloader module is used to extract the raw information about the datasets into the ./data folder

In [2]:
downloader = Downloader()

In [ ]:
downloader.download_and_extract_dataset(OmrDataset.MuscimaPlusPlus_V2, "data/muscima_pp_v2")

In [ ]:
downloader.download_and_extract_dataset(OmrDataset.Homus_V2, "data/homus_v2")

In [3]:
downloader.download_and_extract_dataset(OmrDataset.Audiveris, "data/audiveris")

Extracting Audiveris dataset...


In [5]:
downloader.download_and_extract_dataset(OmrDataset.Capitan, "data/capitan")

Downloading: https://github.com/apacha/OMR-Datasets/releases/download/datasets/BimodalHandwrittenSymbols.zip Bytes: 14535718 into c:\Users\S_CSIS-PostGrad\Documents\NWU\Projek\Recognition-of-Expressive-Symbols-in-Handwritten-Sheet-Music\BimodalHandwrittenSymbols.zip



Extracting Capitan dataset...


The generated raw files are then used to draw the images of the dataset on canvases and store them in different folders.

In [ ]:
homus_generator = HomusImageGenerator()
homus_generator.create_images("./data/homus_v2", "./data/homus_v2_images", [2])

In [ ]:
muscima_generator = MuscimaPlusPlusSymbolImageGenerator()
muscima_generator.extract_and_render_all_symbol_masks("./data/muscima_pp_v2/", "./data/muscima_pp_v2_symbols/")


In [4]:
audiveris_generator = AudiverisOmrImageGenerator()
audiveris_generator.extract_symbols("./data/audiveris", "./data/audiveris_images")

Extracting Symbols from Audiveris OMR Dataset...


In [7]:
capitan_generator = CapitanImageGenerator()
capitan_generator.create_capitan_images("./data/capitan/", "./data/capitan_images", [1])

Loading symbols from strings: 100%|██████████| 10230/10230 [00:01<00:00, 6056.08it/s]

Generating 10230 images with 10230 symbols in 1 different stroke thicknesses ([1])
In directory c:\Users\S_CSIS-PostGrad\Documents\NWU\Projek\Recognition-of-Expressive-Symbols-in-Handwritten-Sheet-Music\data\capitan_images



Rendering strokes: 100%|██████████| 10230/10230 [00:13<00:00, 760.40it/s]

Generating 10230 images from Capitan symbols
In directory c:\Users\S_CSIS-PostGrad\Documents\NWU\Projek\Recognition-of-Expressive-Symbols-in-Handwritten-Sheet-Music\data\capitan_images



Rendering images: 100%|██████████| 10230/10230 [00:16<00:00, 635.68it/s]
